In [85]:
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# 1. Charger le dataset
dataset, info = tfds.load('malaria', as_supervised=True, with_info=True, shuffle_files=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Utilisateur\tensorflow_datasets\malaria\incomplete.JELINY_1.0.0\malaria-train.tfrecord*...:…

Dataset malaria downloaded and prepared to C:\Users\Utilisateur\tensorflow_datasets\malaria\1.0.0. Subsequent calls will reuse this data.


In [86]:
# Informations sur le dataset
print("Description du dataset :")
print(info.description)
print("\nNombre total d'exemples :", info.splits['train'].num_examples)

Description du dataset :
The Malaria dataset contains a total of 27,558 cell images with equal instances
of parasitized and uninfected cells from the thin blood smear slide images of
segmented cells.

Nombre total d'exemples : 27558


In [87]:
# 2. Définir les fractions pour division en entraînement, validation et test
train_split = 0.8
val_split = 0.1
test_split = 0.1

# Diviser les données en sous-ensembles
train_data = dataset['train'].take(int(info.splits['train'].num_examples * train_split))
val_data = dataset['train'].skip(int(info.splits['train'].num_examples * train_split)).take(int(info.splits['train'].num_examples * val_split))
test_data = dataset['train'].skip(int(info.splits['train'].num_examples * (train_split + val_split)))

# Vérification des tailles des datasets
print(f"Exemples d'entraînement : {len(list(train_data))}")
print(f"Exemples de validation : {len(list(val_data))}")
print(f"Exemples de test : {len(list(test_data))}")

Exemples d'entraînement : 22046
Exemples de validation : 2755
Exemples de test : 2756


In [81]:
# 3. Prétraitement des données
batch_size = 32
img_size = (128, 128)  # Taille des images cible

def preprocess_image(image, label):
    """
    Prétraite une image :
    - Redimensionne l'image
    - Normalise les pixels entre 0 et 1
    """
    image = tf.image.resize(image, img_size)
    image = tf.cast(image, tf.float32) / 255.0
    return image

def encode_label(label):
    """
    Encode les labels en format binaire :
    - 1 pour 'Parasitized'
    - 0 pour 'Uninfected'
    """
    return tf.cast(label, tf.float32)  # Assure un format compatible avec les modèles TensorFlow

# Appliquer le prétraitement et l'encodage binaire des labels
def preprocess_and_encode(image, label):
    image = preprocess_image(image, label)
    label = encode_label(label)
    return image, label

train_data = train_data.map(preprocess_and_encode, num_parallel_calls=tf.data.AUTOTUNE)
val_data = val_data.map(preprocess_and_encode, num_parallel_calls=tf.data.AUTOTUNE)
test_data = test_data.map(preprocess_and_encode, num_parallel_calls=tf.data.AUTOTUNE)

# Ajouter batching et préchargement pour optimiser l'entraînement
train_data = train_data.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
val_data = val_data.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
test_data = test_data.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

# 4. Visualiser un échantillon d'images et leurs labels
def visualize_sample(dataset, title, class_names=['Parasitized', 'Uninfected']):
    plt.figure(figsize=(10, 10))
    for images, labels in dataset.take(1):  # Prenez un batch pour visualisation
        for i in range(9):  # Affichez les 9 premières images
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy())
            plt.title(f"Label: {class_names[int(labels[i])]}")  # Convertir le label en entier pour l'affichage
            plt.axis("off")
    plt.suptitle(title, fontsize=16)
    plt.show()

print("\nVisualisation des données d'entraînement :")
visualize_sample(train_data, "Échantillon d'images - Entraînement")

# 5. Répartition des classes
def plot_class_distribution(dataset, dataset_name, class_names=['Parasitized', 'Uninfected']):
    labels = np.concatenate([y.numpy() for _, y in dataset])
    unique, counts = np.unique(labels, return_counts=True)

    plt.figure(figsize=(6, 6))
    plt.pie(
        counts,
        labels=[class_names[int(i)] for i in unique],
        autopct='%1.1f%%',
        colors=['skyblue', 'salmon']
    )
    plt.title(f"Répartition des classes - {dataset_name}")
    plt.show()

print("\nRépartition des classes dans chaque sous-dataset :")
plot_class_distribution(train_data, "Entraînement")
plot_class_distribution(val_data, "Validation")
plot_class_distribution(test_data, "Test")
